# Linear regression, lambda KAN training on MNIST

In [ ]:
import os
import sys

import torch
import torchmetrics
import numpy as np
import torch.nn as nn
import matplotlib.pyplot as plt

from collections import defaultdict
from torch.nn.modules.pooling import MaxPool2d
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, utils
from torchvision.transforms import v2
from torch.utils.data import DataLoader
from IPython.display import clear_output

import torch
import torchmetrics
import lightning as L
import torch.nn as nn
import matplotlib.pyplot as plt

from torchmetrics import MetricCollection
from torchmetrics.classification import (
    MulticlassAccuracy,
    MulticlassF1Score,
    MulticlassAUROC,
)
from lightning.pytorch.callbacks import ModelCheckpoint
from tbparse import SummaryReader

from kan import KANLayer
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath('__file__'))))
from source import lmdKANLayer

import random

from models import MnistLR, MnistMLP, Mnist_lmdSplineKAN, Mnist_tlmdSplineKAN, show_sensitivity

### Load [MNIST](https://pytorch.org/vision/stable/generated/torchvision.datasets.MNIST.html#torchvision.datasets.MNIST):

In [ ]:
batch_size = 256

transform = v2.Compose(
    [
        v2.ToImage(),
        v2.ToDtype(torch.float32, scale=True),
    ]
)

mnist = datasets.MNIST(root="./", train=True, download=True, transform=transform)

(train_set, val_set,) = torch.utils.data.random_split(
    mnist, [50000, 10000], generator=torch.Generator().manual_seed(42)
)
test_set = datasets.MNIST(root="./", train=False, download=True, transform=transform)

val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False, num_workers=2, persistent_workers=True)
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=2, persistent_workers=True)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=2, persistent_workers=True)

### Create LightningModule class

In [ ]:
class LModel(L.LightningModule):
    def __init__(self, model, lr=0.001):
        super().__init__()
        self.save_hyperparameters(logger=False)

        # for optimizer and shaduler
        self.lr = lr

        # model
        self.model = model
        self.criterion = nn.CrossEntropyLoss()

        # metrics
        self.metrics = MetricCollection([
            MulticlassAccuracy(num_classes=10,),
            #MulticlassF1Score(num_classes=10,),
            #MulticlassAUROC(num_classes=10,)
        ])
        self.train_metrics = self.metrics.clone(postfix='/train')
        self.val_metrics = self.metrics.clone(postfix='/val')

    def configure_optimizers(self):
        # set optimizer
        optimizer = torch.optim.Adam(
            self.model.parameters(),
            lr=self.lr,
        )
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer,
        )
        return {
            "optimizer": optimizer,
            "lr_scheduler": {
                "scheduler": scheduler,
                "interval": "epoch",  # or 'step'
                "monitor": "loss" # only for self.log
            },
        }

    def training_step(self, batch, batch_idx):
        x, y = batch
        out = self.model(x)
        loss = self.criterion(out, y)
        self.train_metrics.update(out, y)
        self.log("loss", loss, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        out = self.model(x)
        self.val_metrics.update(out, y)

    def on_train_epoch_end(self):
        self.log_dict(self.train_metrics.compute())
        self.train_metrics.reset()

        val_metrics = self.val_metrics.compute()
        self.log_dict(val_metrics)
        self.val_metrics.reset()

    def test_step(self, batch, batch_idx):
        x, y = batch
        out = self.model(x)
        self.metrics.update(out, y)

    def on_test_epoch_end(self):
        self.log_dict(self.metrics.compute())
        self.metrics.reset()

### Start training

#### MnistLR

In [ ]:
class LModel_save_pic(LModel):
    def on_train_epoch_end(self):
        self.log_dict(self.train_metrics.compute())
        self.train_metrics.reset()
        
        val_metrics = self.val_metrics.compute()
        self.log_dict(val_metrics)
        self.val_metrics.reset()
        
        directory = self.logger.log_dir + '\\sens_pic'
        if not os.path.exists(directory):
            os.makedirs(directory)
        show_sensitivity(W = self.model.get_W(),
                         title = f'Linear Regression model | epoch={self.current_epoch+1}',
                         save_pic=True, 
                         file_path=directory + f'\\{self.current_epoch}.png')

checkpoint_callback = ModelCheckpoint(
    # dirpath='best_ckp/'
    save_last=True,
    every_n_epochs=1,
    save_top_k=1,
    save_weights_only=True,
    monitor="MulticlassAccuracy/val",
    filename="model",
    mode="max",
)

L.seed_everything(1755)

model = MnistLR() #Mnist_lmdSplineKAN() #MnistPrllSplineKAN() #MnistSplineKAN() #MnistSplineKAN() #MnistKAN() #MnistMLP() #MnistCNN() #MnistLR()
pl_model = LModel_save_pic(model)
trainer = L.Trainer(
    max_epochs=20,
    callbacks=[checkpoint_callback],
    num_sanity_val_steps=0,
    log_every_n_steps=10,
    logger=L.pytorch.loggers.TensorBoardLogger(save_dir=f"./logs/{type(model).__name__}/"),
)

trainer.fit(
    model=pl_model,
    train_dataloaders=train_loader,
    val_dataloaders=val_loader
)

In [ ]:
# Load the best checkpoint automatically (lightning tracks this for you)
trainer.test(
    model=pl_model, dataloaders=test_loader, verbose=True, ckpt_path="best"
)

#### MnistMLP

In [ ]:
checkpoint_callback = ModelCheckpoint(
    # dirpath='best_ckp/'
    save_last=True,
    every_n_epochs=1,
    save_top_k=1,
    save_weights_only=True,
    monitor="MulticlassAccuracy/val",
    filename="model",
    mode="max",
)

L.seed_everything(1755)

model = MnistMLP() #Mnist_lmdSplineKAN() #MnistPrllSplineKAN() #MnistSplineKAN() #MnistSplineKAN() #MnistKAN() #MnistMLP() #MnistCNN() #MnistLR()
pl_model = LModel_save_pic(model)
trainer = L.Trainer(
    max_epochs=20,
    callbacks=[checkpoint_callback],
    num_sanity_val_steps=0,
    log_every_n_steps=10,
    logger=L.pytorch.loggers.TensorBoardLogger(save_dir=f"./logs/{type(model).__name__}/"),
)

trainer.fit(
    model=pl_model,
    train_dataloaders=train_loader,
    val_dataloaders=val_loader
)

In [ ]:
# Load the best checkpoint automatically (lightning tracks this for you)
trainer.test(
    model=pl_model, dataloaders=test_loader, verbose=True, ckpt_path="best"
)

#### Mnist_lmdSplineKAN

In [ ]:
class LModel_save_pic(LModel):
    def on_train_epoch_end(self):
        self.log_dict(self.train_metrics.compute())
        self.train_metrics.reset()
        
        val_metrics = self.val_metrics.compute()
        self.log_dict(val_metrics)
        self.val_metrics.reset()
        
        directory = self.logger.log_dir + '\\sens_pic'
        if not os.path.exists(directory):
            os.makedirs(directory)
        show_sensitivity(W = self.model.get_W(),
                         title = f'lambda KAN model | epoch={self.current_epoch+1}',
                         save_pic=True, 
                         file_path=directory + f'\\{self.current_epoch}.png')


checkpoint_callback = ModelCheckpoint(
    # dirpath='best_ckp/'
    save_last=True,
    every_n_epochs=1,
    save_top_k=1,
    save_weights_only=True,
    monitor="MulticlassAccuracy/val",
    filename="model",
    mode="max",
)

L.seed_everything(1755)

model = Mnist_lmdSplineKAN() #Mnist_lmdSplineKAN() #MnistPrllSplineKAN() #MnistSplineKAN() #MnistSplineKAN() #MnistKAN() #MnistMLP() #MnistCNN() #MnistLR()
pl_model = LModel_save_pic(model)
trainer = L.Trainer(
    max_epochs=20,
    callbacks=[checkpoint_callback],
    num_sanity_val_steps=0,
    log_every_n_steps=10,
    logger=L.pytorch.loggers.TensorBoardLogger(save_dir=f"./logs/{type(model).__name__}/"),
)

trainer.fit(
    model=pl_model,
    train_dataloaders=train_loader,
    val_dataloaders=val_loader
)

In [ ]:
# Load the best checkpoint automatically (lightning tracks this for you)
trainer.test(
    model=pl_model, dataloaders=test_loader, verbose=True, ckpt_path="best"
)

#### Mnist_tlmdSplineKAN

In [ ]:
class LModel_save_pic(LModel):
    def on_train_epoch_end(self):
        self.log_dict(self.train_metrics.compute())
        self.train_metrics.reset()
        
        val_metrics = self.val_metrics.compute()
        self.log_dict(val_metrics)
        self.val_metrics.reset()
        
        directory = self.logger.log_dir + '\\sens_pic'
        if not os.path.exists(directory):
            os.makedirs(directory)
        show_sensitivity(W = self.model.get_W(),
                         title = f'trainable lambda KAN model | epoch={self.current_epoch+1}',
                         save_pic=True, 
                         file_path=directory + f'\\{self.current_epoch}.png')

checkpoint_callback = ModelCheckpoint(
    # dirpath='best_ckp/'
    save_last=True,
    every_n_epochs=1,
    save_top_k=1,
    save_weights_only=True,
    monitor="MulticlassAccuracy/val",
    filename="model",
    mode="max",
)

L.seed_everything(1755)

model = Mnist_tlmdSplineKAN() #Mnist_lmdSplineKAN() #MnistPrllSplineKAN() #MnistSplineKAN() #MnistSplineKAN() #MnistKAN() #MnistMLP() #MnistCNN() #MnistLR()
pl_model = LModel(model)
trainer = L.Trainer(
    max_epochs=20,
    callbacks=[checkpoint_callback],
    num_sanity_val_steps=0,
    log_every_n_steps=10,
    logger=L.pytorch.loggers.TensorBoardLogger(save_dir=f"./logs/{type(model).__name__}/"),
)

trainer.fit(
    model=pl_model,
    train_dataloaders=train_loader,
    val_dataloaders=val_loader
)

In [ ]:
# Load the best checkpoint automatically (lightning tracks this for you)
trainer.test(
    model=pl_model, dataloaders=test_loader, verbose=True, ckpt_path="best"
)

### Plot training curve, if needed

In [ ]:
log_dir_LR = 'logs\\MnistLR\\lightning_logs\\version_0'
log_dir_MLP = 'logs\\MnistMLP\\lightning_logs\\version_0'
log_dir_lmdSplineKAN = 'logs\\Mnist_lmdSplineKAN\\lightning_logs\\version_0'
log_dir_tlmdSplineKAN = 'logs\\Mnist_tlmdSplineKAN\\lightning_logs\\version_0'

l_dirs = [log_dir_LR, log_dir_MLP, log_dir_lmdSplineKAN, log_dir_tlmdSplineKAN]
l_names = ['Linear Regression', 'MLP', 'lambda KAN', 'trainable lambda KAN']

In [ ]:
plt.figure(figsize=(10, 3))

for name, log_dir in zip(l_names, l_dirs):
    reader = SummaryReader(log_dir)
    df = reader.scalars

    def drop_nan(df, tag):
        return df[~df[tag].isna()].loc[:, tag]

    acc = drop_nan(df, "MulticlassAccuracy/val").values
    plt.plot(list(range(1, len(acc)+1)), acc, label=name)


plt.xlabel('Epoch number')
plt.ylabel('Validation multiclass accuracy')
plt.title("Training curve of validation accuracy")
plt.grid()
plt.legend(loc="lower right")
plt.show()